In [1]:
#  imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [2]:
# load data
df=pd.read_csv('ALOI_norm.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att19,att20,att21,att22,att23,att24,att25,att26,att27,outlier
0,0.784999,0.000018,0.0,0.000093,0.0,0.0,0.0,0.0,0.0,0.139811,...,0.048171,0.001189,0.0,0.115728,0.023449,0.0002,0.000220,0.034952,0.046914,1
1,0.958088,0.000000,0.0,0.001671,0.0,0.0,0.0,0.0,0.0,0.019556,...,0.000000,0.000000,0.0,0.022035,0.007516,0.0000,0.000000,0.001033,0.000000,1
2,0.938768,0.000000,0.0,0.005146,0.0,0.0,0.0,0.0,0.0,0.018451,...,0.000000,0.000000,0.0,0.035542,0.011982,0.0000,0.001595,0.019520,0.000000,1
3,0.954775,0.000000,0.0,0.001427,0.0,0.0,0.0,0.0,0.0,0.024944,...,0.000000,0.000000,0.0,0.019941,0.000805,0.0000,0.000000,0.000035,0.000000,1
4,0.933601,0.000000,0.0,0.001682,0.0,0.0,0.0,0.0,0.0,0.037002,...,0.000000,0.000000,0.0,0.046759,0.002663,0.0000,0.000339,0.001359,0.000000,1


In [3]:
df.shape

(50000, 28)

In [4]:
# df to values
df = df.values

In [5]:
# Deep ExtraTrees
import argparse
import numpy as np
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/aloi.json")  
gc = GCForest(config)

In [7]:
# train test
from sklearn.cross_validation import train_test_split
y = df[:,27]
X = df[:,0:27]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
# count class
len(np.unique(y_train))

2

In [8]:
  # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
  

[ 2018-04-25 01:09:54,224][cascade_classifier.fit_transform] X_groups_train.shape=[(35000, 27)],y_train.shape=(35000,),X_groups_test.shape=[(15000, 27)],y_test.shape=(15000,)
[ 2018-04-25 01:09:54,227][cascade_classifier.fit_transform] group_dims=[27]
[ 2018-04-25 01:09:54,228][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:09:54,230][cascade_classifier.fit_transform] group_ends=[27]
[ 2018-04-25 01:09:54,232][cascade_classifier.fit_transform] X_train.shape=(35000, 27),X_test.shape=(15000, 27)
[ 2018-04-25 01:09:54,239][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(35000, 27), X_cur_test.shape=(15000, 27)
[ 2018-04-25 01:09:55,112][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.20%
[ 2018-04-25 01:09:55,974][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.23%
[ 2018-04-25 01:09:56,791][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtratTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 01:10:03,424][cascade_classifier.transform] X_groups_test.shape=[(15000, 27)]
[ 2018-04-25 01:10:03,428][cascade_classifier.transform] group_dims=[27]
[ 2018-04-25 01:10:03,430][cascade_classifier.transform] X_test.shape=(15000, 27)
[ 2018-04-25 01:10:03,433][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(15000, 27)


Test Accuracy of ExtratTrees = 97.260000 %


In [10]:
#  predict_proba + Data 
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 01:10:06,267][cascade_classifier.transform] X_groups_test.shape=[(15000, 27)]
[ 2018-04-25 01:10:06,270][cascade_classifier.transform] group_dims=[27]
[ 2018-04-25 01:10:06,272][cascade_classifier.transform] X_test.shape=(15000, 27)
[ 2018-04-25 01:10:06,275][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(15000, 27)


In [11]:
# New data
X_train_new = X_train_enc
X_test_new  = X_test_enc

In [12]:
X_test_new.shape

(15000, 29)

In [13]:
X_train_new.shape

(35000, 29)

In [14]:
# Cascade ExtraTrees
config = load_json("./examples/aloigc.json")  
gc = GCForest(config)

X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 01:10:07,692][cascade_classifier.fit_transform] X_groups_train.shape=[(35000, 29)],y_train.shape=(35000,),X_groups_test.shape=[(15000, 29)],y_test.shape=(15000,)
[ 2018-04-25 01:10:07,697][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-04-25 01:10:07,698][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:10:07,699][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-04-25 01:10:07,701][cascade_classifier.fit_transform] X_train.shape=(35000, 29),X_test.shape=(15000, 29)
[ 2018-04-25 01:10:07,707][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(35000, 29), X_cur_test.shape=(15000, 29)
[ 2018-04-25 01:10:08,428][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.80%
[ 2018-04-25 01:10:09,315][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.57%
[ 2018-04-25 01:10:10,298][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

[ 2018-04-25 01:10:46,852][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=97.86%
[ 2018-04-25 01:10:47,859][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=97.91%
[ 2018-04-25 01:10:48,738][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=97.69%
[ 2018-04-25 01:10:49,610][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=97.71%
[ 2018-04-25 01:10:50,552][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=98.03%
[ 2018-04-25 01:10:51,418][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.03%
[ 2018-04-25 01:10:52,411][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.74%
[ 2018-04-25 01:10:53,379][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=98.09%


In [15]:
# deep
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of Extratrees = {:.6f} %".format(acc * 100),i,'fois')

In [16]:
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    
    config = load_json("./examples/aloigc.json")  
    gc = GCForest(config)
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()    

[ 2018-04-25 01:11:21,963][cascade_classifier.fit_transform] X_groups_train.shape=[(35000, 29)],y_train.shape=(35000,),X_groups_test.shape=[(15000, 29)],y_test.shape=(15000,)
[ 2018-04-25 01:11:21,967][cascade_classifier.fit_transform] group_dims=[29]
[ 2018-04-25 01:11:21,968][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 01:11:21,970][cascade_classifier.fit_transform] group_ends=[29]
[ 2018-04-25 01:11:21,971][cascade_classifier.fit_transform] X_train.shape=(35000, 29),X_test.shape=(15000, 29)
[ 2018-04-25 01:11:21,976][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(35000, 29), X_cur_test.shape=(15000, 29)
[ 2018-04-25 01:11:22,686][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.71%
[ 2018-04-25 01:11:23,552][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.51%
[ 2018-04-25 01:11:24,419][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estim

[ 2018-04-25 01:11:59,932][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_1.predict)=97.83%
[ 2018-04-25 01:12:00,844][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_2.predict)=97.71%
[ 2018-04-25 01:12:01,756][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_3.predict)=97.49%
[ 2018-04-25 01:12:02,692][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_4.predict)=97.63%
[ 2018-04-25 01:12:03,585][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_5.predict)=98.06%
[ 2018-04-25 01:12:04,409][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=97.91%
[ 2018-04-25 01:12:05,315][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.74%
[ 2018-04-25 01:12:06,157][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=97.89%


('Test Accuracy of Extratrees = 97.986667 %', 0, 'fois')


[ 2018-04-25 01:12:11,688][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.60%
[ 2018-04-25 01:12:12,633][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.43%
[ 2018-04-25 01:12:13,679][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.71%
[ 2018-04-25 01:12:14,539][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.71%
[ 2018-04-25 01:12:15,376][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.54%
[ 2018-04-25 01:12:16,250][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.86%
[ 2018-04-25 01:12:17,116][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.94%
[ 2018-04-25 01:12:18,007][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.83%


[ 2018-04-25 01:12:53,856][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=98.09%
[ 2018-04-25 01:12:54,791][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.66%
[ 2018-04-25 01:12:55,710][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=97.80%
[ 2018-04-25 01:12:56,716][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=97.80%
[ 2018-04-25 01:12:56,864][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=97.81%
[ 2018-04-25 01:12:56,866][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.87%
[ 2018-04-25 01:12:56,871][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=97.81%
[ 2018-04-25 01:12:56,872][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.87%
[ 2018-04-25 01:12:56,886][

...

[ 2018-04-25 01:35:22,017][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.37%
[ 2018-04-25 01:35:22,908][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.46%
[ 2018-04-25 01:35:23,787][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=97.51%
[ 2018-04-25 01:35:24,658][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.57%
[ 2018-04-25 01:35:25,530][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.46%
[ 2018-04-25 01:35:26,414][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=97.60%
[ 2018-04-25 01:35:27,454][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.86%
[ 2018-04-25 01:35:28,383][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.26%


[ 2018-04-25 01:36:03,145][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_6.predict)=97.60%
[ 2018-04-25 01:36:04,097][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_7.predict)=97.34%
[ 2018-04-25 01:36:05,025][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_8.predict)=97.66%
[ 2018-04-25 01:36:06,045][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_9.predict)=97.43%
[ 2018-04-25 01:36:06,181][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.train_cv.predict)=97.55%
[ 2018-04-25 01:36:06,184][kfold_wrapper.log_eval_metrics] Accuracy(layer_4 - estimator_0 - 10_folds.test.predict)=97.54%
[ 2018-04-25 01:36:06,188][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=97.55%
[ 2018-04-25 01:36:06,190][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=97.54%
[ 2018-04-25 01:36:06,202][

[ 2018-04-25 01:36:41,744][cascade_classifier.calc_accuracy] Accuracy(layer_8 - train.classifier_average)=97.52%
[ 2018-04-25 01:36:41,746][cascade_classifier.calc_accuracy] Accuracy(layer_8 - test.classifier_average)=97.53%
[ 2018-04-25 01:36:41,757][cascade_classifier.fit_transform] [layer=9] look_indexs=[0], X_cur_train.shape=(35000, 31), X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:36:42,492][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_0.predict)=97.51%
[ 2018-04-25 01:36:43,431][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_1.predict)=97.71%
[ 2018-04-25 01:36:44,342][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_2.predict)=97.51%
[ 2018-04-25 01:36:45,211][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_3.predict)=97.57%
[ 2018-04-25 01:36:46,068][kfold_wrapper.log_eval_metrics] Accuracy(layer_9 - estimator_0 - 10_folds.train_4.predict)=97.63%
[ 

[ 2018-04-25 01:37:20,007][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:22,792][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:25,588][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:28,392][cascade_classifier.transform] [layer=4] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:31,912][cascade_classifier.transform] [layer=5] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:35,587][cascade_classifier.transform] [layer=6] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:39,262][cascade_classifier.transform] [layer=7] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:42,641][cascade_classifier.transform] [layer=8] look_indexs=[0], X_cur_test.shape=(15000, 31)
[ 2018-04-25 01:37:45,895][cascade_classifier.transform] [layer=9] look_indexs=[0], X_cur_test.s

('Test Accuracy of Extratrees = 97.520000 %', 19, 'fois')


# ('Test Accuracy of Extratrees = 97.986667 %', 0, 'fois')